### Taxi - v3 RL solution ###


In [ ]:
import gymnasium as gym

import numpy as np
import random
from IPython.display import clear_output
from time import sleep

In [ ]:
# Setting up an environment

env = gym.make("Taxi-v3", render_mode='ansi')

In [ ]:
print(env.action_space,env.observation_space)
print(env.action_space.sample())
print(env.observation_space.sample())

Discrete(6) Discrete(500)
1
496


In [ ]:
env.reset()
result = env.step(0)
result

(212,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})

In [ ]:
# Set up the Q-table

q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(q_table.shape)


(500, 6)


In [ ]:
%%time
"""Training the agent"""

frames = []

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()[0]

    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info, _ = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(),
            'state': state,
            'action': action,
            'reward': reward
        })
        state = next_state
        epochs += 1


    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")


Episode: 100000
Training finished.

CPU times: user 33.7 s, sys: 7.48 s, total: 41.1 s
Wall time: 35.9 s


In [ ]:
# # Run the frames to see how the agent did over the episodes

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        print(f"Episode: {frame['episode']}")
        sleep(.1)

# print_frames(frames)

In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 5
framed = []

for e in range(episodes):
    state = env.reset()[0]
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info, _ = env.step(action)

        if reward == -10:
            penalties += 1

        # Put each rendered frame into dict for animation
        framed.append({
            'frame': env.render(),
            'state': state,
            'action': action,
            'reward': reward,
            "episode":e
        })
        print_frames(framed)
        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 63
State: 0
Action: 5
Reward: 20
Episode: 4
Results after 5 episodes:
Average timesteps per episode: 12.6
Average penalties per episode: 0.0
